## 혼합 검색 기능 활용 
- 의미 검색(벡터기반)과 키워드 검색(통계기반) 혼합
- RRF(상호순위조합, Reciprcal Rank Fusion) 작성

In [1]:
import math
import numpy as np
from typing import List
from transformers import PreTrainedTokenizer
from collections import defaultdict

In [29]:
# RRF 함수 작성
# 각 순위 점수 = 1 / (K + 순위)
from collections import defaultdict

def reciprocal_rank_fusion(rankings:List[List[int]], k=5): # 실제로는 k를 조금 더 크게 약 60 정도
    rrf = defaultdict(float) # float으로 초기화 
    for ranking in rankings:
        for i, doc_id in enumerate(ranking, 1):
            rrf[doc_id] += 1.0 / (k + i)
            
    return sorted(rrf.items(), key=lambda x: x[1], reverse=True)

In [3]:
rank_list = [[1,4,3,5,6] # 의미 검색 순위, 각 문장의 인덱스
             ,[2,1,3,6,4]] # 통계 검색 순위, 각 문장의 인덱스

reciprocal_rank_fusion(rank_list)

[(1, 0.30952380952380953),
 (3, 0.25),
 (4, 0.24285714285714285),
 (6, 0.2111111111111111),
 (2, 0.16666666666666666),
 (5, 0.1111111111111111)]

In [4]:
reciprocal_rank_fusion(rank_list, k=40 )

[(1, 0.04819976771196283),
 (3, 0.046511627906976744),
 (4, 0.046031746031746035),
 (6, 0.04494949494949495),
 (2, 0.024390243902439025),
 (5, 0.022727272727272728)]

### 의미 검색

In [5]:
# https://huggingface.co/datasets/klue/klue/viewer/mrc
from datasets import load_dataset

klue_mrc_dataset = load_dataset('klue', 'mrc', split='train')

README.md:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17554 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5841 [00:00<?, ? examples/s]

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [7]:
from sentence_transformers import SentenceTransformer

model_sentence = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")
model_sentence

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [8]:
embeddings = model_sentence.encode(klue_mrc_dataset['context'])
embeddings

Batches:   0%|          | 0/549 [00:00<?, ?it/s]

array([[ 0.5222375 , -1.1391711 ,  0.12654702, ..., -1.1193508 ,
         0.03521788, -0.63249075],
       [-0.36406726, -0.63196135,  0.190007  , ..., -0.21136642,
         0.40211734,  0.38462615],
       [-0.36406726, -0.63196135,  0.190007  , ..., -0.21136642,
         0.40211734,  0.38462615],
       ...,
       [-0.93149436,  0.02786567, -0.5193919 , ...,  0.4105615 ,
        -0.03118349, -0.13073075],
       [-0.16274337, -0.47562665, -0.633726  , ...,  0.15545923,
        -0.39259574, -0.46845305],
       [-0.9013975 , -1.1034583 ,  0.189643  , ..., -0.35165054,
        -0.09077406,  0.37111798]], dtype=float32)

In [9]:
!pip install faiss-cpu faiss-gpu -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 56.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 20.8 MB/s eta 0:00:00:00:0100:01


In [10]:
import faiss # meta api : vector distance 

index_KNN = faiss.IndexFlatL2(embeddings.shape[1]) # KNN 알고리즘 초기화, vocab 사이즈 만큼 vector 공간 할당
index_KNN

<faiss.swigfaiss_avx512.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7f67a6c8a580> >

In [11]:
index_KNN.add(embeddings)


### 통계 구현

In [12]:
class BM25:
    
    def __init__(self, corpus: List[List[str]], tokenizer: PreTrainedTokenizer):
        # Initialize BM25 with a list of tokenized documents and a tokenizer.
        self.tokenizer = tokenizer
        self.corpus = corpus
        
        # Tokenize the entire corpus. This converts words into token IDs.
        self.tokenized_corpus = self.tokenizer(corpus, add_special_tokens=False)['input_ids']
        
        # Number of documents in the corpus.
        self.n_docs = len(self.tokenized_corpus)
        
        # Calculate the average document length in tokens.
        self.avg_doc_lens = sum(len(doc) for doc in self.tokenized_corpus) / self.n_docs
        
        # Compute the Inverse Document Frequency (IDF) values.
        self.idf = self._calculate_idf()
        
        # Compute the term frequencies for each document.
        self.term_freqs = self._calculate_term_freqs()
        
    def _calculate_idf(self):
        # Calculate Inverse Document Frequency (IDF) for each unique token in the corpus.
        idf = defaultdict(float)
        
        # Count the number of documents containing each token.
        for doc in self.tokenized_corpus:
            for token_id in set(doc):
                idf[token_id] += 1
                
        # Apply the BM25-specific IDF formula for each token.
        for token_id, doc_frequency in idf.items():
            idf[token_id] = math.log(((self.n_docs - doc_frequency + 0.5) / (doc_frequency + 0.5)) + 1)
            
        return idf
        
    def _calculate_term_freqs(self):
        # Compute the frequency of each token in each document.
        term_freqs = [defaultdict(int) for _ in range(self.n_docs)]
        
        for i, doc in enumerate(self.tokenized_corpus):
            for token_id in doc:
                term_freqs[i][token_id] += 1
        
        return term_freqs
        
    def get_scores(self, query: str, k1: float = 1.2, b: float = 0.75):
        # Calculate BM25 scores for all documents given a query.
        # k1 controls term frequency saturation; b adjusts document length normalization.
        query = self.tokenizer([query], add_special_tokens=False)['input_ids'][0]
        scores = np.zeros(self.n_docs)
        
        # Compute BM25 scores for each query token.
        for q in query:
            idf = self.idf[q]  # Retrieve the precomputed IDF for the query token.
            
            for i, term_freq in enumerate(self.term_freqs):
                q_frequency = term_freq[q]  # Term frequency of the query token in the current document.
                doc_len = len(self.tokenized_corpus[i])
                
                # BM25 formula to compute the score contribution of this token.
                score_q = idf * (q_frequency * (k1 + 1)) / (q_frequency + k1 * (1 - b + b * (doc_len / self.avg_doc_lens)))
                
                # Accumulate the score for document i.
                scores[i] += score_q
                
        return scores
        
    def get_top_k(self, query: str, k: int):
        # Get the top-k documents based on BM25 scores for the given query.
        scores = self.get_scores(query)
        # Sort document indices by scores in descending order and select top-k.
        top_k_indices = np.argsort(scores)[-k:][::-1]
        # Retrieve the scores for the top-k documents.
        top_k_scores = scores[top_k_indices]
        
        return top_k_scores, top_k_indices

In [13]:
index_bm25 = BM25(klue_mrc_dataset['context'], tokenizer)
index_bm25 

Token indices sequence length is longer than the specified maximum sequence length for this model (965 > 512). Running this sequence through the model will result in indexing errors


### 합산

In [57]:
query = '이번 연도에는 언제 비가 많이 올까?'
search_num = 50

In [65]:
def hybrid_RRF_search(query, search_num):
    # 의미 검색
    embedding_query = model_sentence.encode([query])
    distances, indices = index_KNN.search(embedding_query, search_num)
    # 통계 검색 
    top_scores, top_indices = index_bm25.get_top_k(query, search_num)

    rank_list = [indices[0], top_indices]
    result_list = reciprocal_rank_fusion(rank_list, k=50 )
    return result_list
    

In [68]:
result_list = hybrid_RRF_search(query, search_num)
len(result_list), result_list[:5]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(99,
 [(9205, 0.03464755077658303),
  (8704, 0.0196078431372549),
  (1326, 0.0196078431372549),
  (8705, 0.019230769230769232),
  (1327, 0.019230769230769232)])

In [67]:
[klue_mrc_dataset['context'][idx[0]][:50] for idx in result_list[:5]]


['다음달엔 평년에 비해 때 이른 무더위가 기승을 부릴 전망이다. 8월에는 대기불안정과 저기압',
 '올 들어 한반도 날씨가 수상쩍다. 23일 하루 동안 서울 등 중부지방엔 호우특보와 폭염특보',
 '케이팝 팬덤을 위한 어플리케이션 ‘블립’ 조사 결과, NCT 팬들이 가장 많이 입덕한 노래',
 '올 들어 한반도 날씨가 수상쩍다. 23일 하루 동안 서울 등 중부지방엔 호우특보와 폭염특보',
 '케이팝 팬덤을 위한 어플리케이션 ‘블립’ 조사 결과, NCT 팬들이 가장 많이 입덕한 노래']

In [41]:
embedding_query = model_sentence.encode([query])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [42]:
distances, indices = index_KNN.search(embedding_query, search_num)
distances, indices

(array([[283.32172, 283.32172, 294.21136, 297.22064, 300.919  , 311.6458 ,
         311.6458 , 318.64502, 321.1699 , 322.0254 , 329.99564, 332.50632,
         332.50632, 332.50632, 335.51825, 335.51825, 339.3164 , 341.31927,
         341.31927, 343.26062, 346.96033, 353.8723 , 355.13086, 357.27625,
         357.27625, 362.27866, 365.65628, 367.0719 , 367.0719 , 379.49274,
         380.04266, 381.67676, 381.67676, 386.86707, 386.86707, 388.9942 ,
         388.9942 , 390.08545, 390.14368, 390.14368, 390.732  , 391.00586,
         391.442  , 391.48416, 391.96515, 392.22168, 392.61487, 392.61487,
         392.91174, 393.96265]], dtype=float32),
 array([[ 8704,  8705, 16294,  9205,     0,  2962,  2963, 16341,  5237,
          7038, 16788,  3014,  3015,  3016, 11780, 11781,  2961,  5464,
          5465,  6694,  1582, 12263,  4906,  3973,  3974,  6819, 17079,
         10175, 10176, 13887, 10286, 12438, 12439,  1419,  1420, 13823,
         13824, 11834, 10062, 10063,  9096, 16796, 12581, 13885

In [69]:
for idx in indices[0][:5]:
    print(klue_mrc_dataset['context'][idx][:50]) # 학습 데이터가 많아야 성능 좋다. 꼭 의미가 안맞아도 거리 가까운게 높게 나온다 
    print('--------------------------------')

올 들어 한반도 날씨가 수상쩍다. 23일 하루 동안 서울 등 중부지방엔 호우특보와 폭염특보
--------------------------------
올 들어 한반도 날씨가 수상쩍다. 23일 하루 동안 서울 등 중부지방엔 호우특보와 폭염특보
--------------------------------
이번 주말부터 전국에 30도를 웃도는 초여름 날씨가 다시 찾아올 전망이다.기상청은 전국이 
--------------------------------
다음달엔 평년에 비해 때 이른 무더위가 기승을 부릴 전망이다. 8월에는 대기불안정과 저기압
--------------------------------
올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 
--------------------------------


In [44]:
index_bm25.get_scores(query)

array([7.33257838, 1.90498688, 1.90498688, ..., 0.05748653, 2.69397309,
       2.10482253])

In [45]:
top_scores, top_indices = index_bm25.get_top_k(query,search_num)
top_scores, top_indices

(array([12.86092752, 12.86092752, 12.18243497, 11.97401308, 11.97401308,
        11.64259151, 11.19571749, 10.63290469, 10.4488602 , 10.36894421,
        10.32097785, 10.25938389, 10.19856   , 10.00688207,  9.98369629,
         9.98369629,  9.73323901,  9.73323901,  9.72845109,  9.37027666,
         9.15937605,  9.09518406,  9.09518406,  8.83239502,  8.83239502,
         8.82280296,  8.76278968,  8.74457502,  8.59240755,  8.59240755,
         8.58265806,  8.58265806,  8.52627589,  8.524531  ,  8.52061145,
         8.45986027,  8.38313481,  8.31565878,  8.29840514,  8.18386897,
         8.16113596,  8.16113596,  8.10140982,  8.10140982,  8.09740735,
         8.09613591,  8.08177112,  8.05106202,  8.02969059,  8.02969059]),
 array([ 1326,  1327,  8018, 13956, 13955, 14436, 11996,  8472, 12648,
        16096, 11314,  9205, 11118, 11264,  7548,  7547,  1265,  1266,
        13827,  5235,    72,  6632,  6631, 13754, 13755,   232,  3089,
          237,  3308,  3309,   274,   273, 14590, 12541

In [70]:
[klue_mrc_dataset['context'][idx][:50] for idx in top_indices[:5]]   

['케이팝 팬덤을 위한 어플리케이션 ‘블립’ 조사 결과, NCT 팬들이 가장 많이 입덕한 노래',
 '케이팝 팬덤을 위한 어플리케이션 ‘블립’ 조사 결과, NCT 팬들이 가장 많이 입덕한 노래',
 '‘현대무용 같지 않다.’ 오는 15일까지 서울 서초동 예술의전당 자유소극장 무대에 오르는 ',
 '알츠하이머 질환인 치매는 65세 이상 노인에게서 많이 발병하는 퇴행성 뇌질환이다. 뇌세포가',
 '알츠하이머 질환인 치매는 65세 이상 노인에게서 많이 발병하는 퇴행성 뇌질환이다. 뇌세포가']

In [48]:
rank_list = [indices.tolist()[0],top_indices.tolist()]

In [50]:
result_list =  reciprocal_rank_fusion(rank_list, k=40 )

In [71]:
result_list[:5]

[(9205, 0.03464755077658303),
 (8704, 0.0196078431372549),
 (1326, 0.0196078431372549),
 (8705, 0.019230769230769232),
 (1327, 0.019230769230769232)]

In [53]:
[klue_mrc_dataset['context'][idx[0]][:50] for idx in result_list]

['다음달엔 평년에 비해 때 이른 무더위가 기승을 부릴 전망이다. 8월에는 대기불안정과 저기압',
 '올 들어 한반도 날씨가 수상쩍다. 23일 하루 동안 서울 등 중부지방엔 호우특보와 폭염특보',
 '케이팝 팬덤을 위한 어플리케이션 ‘블립’ 조사 결과, NCT 팬들이 가장 많이 입덕한 노래',
 '올 들어 한반도 날씨가 수상쩍다. 23일 하루 동안 서울 등 중부지방엔 호우특보와 폭염특보',
 '케이팝 팬덤을 위한 어플리케이션 ‘블립’ 조사 결과, NCT 팬들이 가장 많이 입덕한 노래',
 '이번 주말부터 전국에 30도를 웃도는 초여름 날씨가 다시 찾아올 전망이다.기상청은 전국이 ',
 '‘현대무용 같지 않다.’ 오는 15일까지 서울 서초동 예술의전당 자유소극장 무대에 오르는 ',
 '알츠하이머 질환인 치매는 65세 이상 노인에게서 많이 발병하는 퇴행성 뇌질환이다. 뇌세포가',
 '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 ',
 '알츠하이머 질환인 치매는 65세 이상 노인에게서 많이 발병하는 퇴행성 뇌질환이다. 뇌세포가',
 '8월 11일에는 조선민주주의인민공화국으로부터 장마전선이 남하하게 됨에 따라, 남부 지방에서',
 '내셔널리그 홈페이지 및 코레일 축구단에서는 공식 창단 연도를 1943년이라고 공표하고 있지',
 '8월 11일에는 조선민주주의인민공화국으로부터 장마전선이 남하하게 됨에 따라, 남부 지방에서',
 '연봉 5억원 이상을 받아 보수를 공개해야 하는 등기임원에 해당 연도에 퇴임한 임원도 포함된',
 '18년 만에 발생한 ‘슈퍼 엘니뇨’ 현상으로 중부지방이 사상 최악의 가뭄에 시달리고 있다.',
 '코로나로 인해 모두가 힘들었던 2020년이 어느 새 끝나가면서, 새로운 한 해를 향한 기대',
 '비록 적도에 위치하고 있기는 하지만, 훔볼트 해류의 영향으로 차가운 바닷물을 섬 주위로 가',
 '지난해 은행권의 주택담보대출 중 고정금리 대출 비중이 크게 증가했다. 주택금융공사의 적격대',
 '한반도 기후가 